In [ ]:
import pandas as pd
import numpy as np
import pickle

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

%matplotlib inline

In [ ]:
# Remove unrelated columns form data and get their name

folder_path = '../../../datalcdem/data/optima/dementia_18July/'

patient_df = pd.read_csv(folder_path + 'optima_patients.csv')
display(patient_df.head(5))

#patient_df[['MMS1', 'MMS2']].hist()

patient_com_df = pd.read_csv(folder_path + 'optima_patients_comorbidities.csv').groupby(by=['patient_id', 'EPISODE_DATE'], as_index=False).agg(lambda x: x.tolist())[['patient_id', 'EPISODE_DATE', 'Comorbidity_cui']]
display(patient_com_df.head(10))

patient_filt_df = pd.read_csv(folder_path + 'optima_patients_filtered.csv')
#display(patient_filt_df.head(5))


patient_treat_df = pd.read_csv(folder_path + 'optima_patients_treatments.csv').groupby(by=['patient_id', 'EPISODE_DATE'], as_index=False).agg(lambda x: x.tolist())[['patient_id', 'EPISODE_DATE', 'Medication_cui']]
display(patient_treat_df.head(5))

In [ ]:
len(set(patient_com_df['patient_id'].tolist())), len(set(patient_treat_df['patient_id'].tolist()))

In [ ]:
patient_treat_df['EPISODE_DATE'] = pd.to_datetime(patient_treat_df['EPISODE_DATE'])
patient_com_df['EPISODE_DATE'] = pd.to_datetime(patient_com_df['EPISODE_DATE'])

patient_com_treat_df = pd.merge(patient_com_df, patient_treat_df,on=['patient_id', 'EPISODE_DATE'], how='outer')
#pd.concat([patient_com_df, patient_treat_df], keys=['patient_id', 'EPISODE_DATE'], ignore_index=True, sort=False)  #patient_com_df.append(patient_treat_df, sort=False)  #pd.concat([patient_com_df, patient_treat_df], axis=0, sort=False)
#patient_treat_com_df = patient_treat_df.join(patient_com_df, on=['patient_id', 'EPISODE_DATE'], how='outer')
print (patient_com_treat_df.shape)
print (len(set(patient_com_treat_df['patient_id'].tolist())))
patient_com_treat_df.sort_values(by=['patient_id', 'EPISODE_DATE'],axis=0, inplace=True, ascending=True)
patient_com_treat_df.reset_index(drop=True, inplace=True)
patient_com_treat_df.head(10)

In [ ]:
patient_com_treat_df.to_csv('../../../datalcdem/data/optima/optima_ahmad/patient_com_treat_df.csv')

In [ ]:
folder_path = '../../../datalcdem/data/optima/dementia_18July/'

In [ ]:
df_datarequest = pd.read_excel(folder_path+'Data_Request_Jan_2019_final.xlsx')
df_datarequest.head(5)
df_datarequest_mmse = df_datarequest[['GLOBAL_PATIENT_DB_ID', 'Age At Episode', 'EPISODE_DATE', 'CAMDEX SCORES: MINI MENTAL SCORE']]
df_datarequest_mmse_1 = df_datarequest_mmse.rename(columns={'GLOBAL_PATIENT_DB_ID':'patient_id'})
df_datarequest_mmse_1.head(10)

In [ ]:
#patient_com_treat_df.astype('datetime')
patient_com_treat_df['EPISODE_DATE'] = pd.to_datetime(patient_com_treat_df['EPISODE_DATE'])
print (df_datarequest_mmse_1.dtypes, patient_com_treat_df.dtypes)
patient_com_treat_df = pd.merge(patient_com_treat_df,df_datarequest_mmse_1,on=['patient_id', 'EPISODE_DATE'], how='left')
patient_com_treat_df.shape, patient_com_treat_df.head(10)

In [ ]:
len(set (patient_com_treat_df['patient_id'].tolist()))
patient_com_treat_df.sort_values(by=['patient_id', 'EPISODE_DATE'],axis=0, inplace=True, ascending=True)
patient_com_treat_df.head(20)
patient_com_treat_df.reset_index(inplace=True, drop=True)
patient_com_treat_df.head(5)

In [ ]:
def setLineNumber(lst):
    lst_dict = {ide:0 for ide in lst}
    lineNumber_list = []
    
    for idx in lst:
        if idx in lst_dict:
           lst_dict[idx] = lst_dict[idx] + 1 
           lineNumber_list.append(lst_dict[idx])
            
    return lineNumber_list
    

patient_com_treat_df['lineNumber'] = setLineNumber(patient_com_treat_df['patient_id'].tolist())
patient_com_treat_df.tail(20)

In [ ]:
df = patient_com_treat_df

id_dict = {i:0 for i in df['patient_id'].tolist()}
for x in df['patient_id'].tolist():
    if x in id_dict:
        id_dict[x]=id_dict[x]+1

line_updated = [int(j) for i in id_dict.values() for j in range(1,i+1)]
print (line_updated[0:10])
df.update(pd.Series(line_updated, name='lineNumber'),errors='ignore')
display(df.head(20))


#patients merging based on id and creating new columns
r = df['lineNumber'].max()
print ('Max line:',r)
l = [df[df['lineNumber']==i] for i in range(1, int(r+1))]
print('Number of Dfs to merge: ',len(l))
df_new = pd.DataFrame()
tmp_id = []
for i, df_l in enumerate(l):
    df_l = df_l[~df_l['patient_id'].isin(tmp_id)]
    for j, df_ll in enumerate(l[i+1:]):
        #df_l = df_l.merge(df_ll, on='id', how='left', suffix=(str(j), str(j+1))) #suffixe is not working
        #print (j)
        df_l = df_l.join(df_ll.set_index('patient_id'), on='patient_id', rsuffix='_'+str(j+1))
    tmp_id = tmp_id + df_l['patient_id'].tolist()
    #display(df_l)
    df_new = df_new.append(df_l, ignore_index=True, sort=False)
display(df_new.head(20))
display(df_new[['patient_id']+[col for col in df_new.columns if 'line' in col or 'DATE' in col]].head(10))

In [ ]:
fltr_linnum = ['_'+str(i) for i in range(10, 27)]
print (fltr_linnum)
df_new.drop(columns=[col for col in df_new.columns for i in fltr_linnum if i in col],inplace=True)

In [ ]:
df_new.to_csv(folder_path+'dementialTreatmentLine_preData_line_episode.csv', index=False)
df_new = df_new.drop([col for col in df_new.columns if 'lineNumber' in col], axis=1).reset_index(drop=True)
df_new.to_csv(folder_path+'dementialTreatmentLine_preData.csv', index=False)

In [ ]:
# Calculate matching intial episode in the data

''' df_episode= pd.read_csv('../../../datalcdem/data/optima/dementialTreatmentLine_preData_line_episode.csv')
df_patients = pd.read_csv('../../../datalcdem/data/optima/patients.csv')
display(df_episode.columns, df_patients.columns)

df_pat_ep = pd.merge(df_episode[['patient_id', 'EPISODE_DATE']], df_patients[['patient_id', 'epDateInicial', 'mmseInicial']])

df_episode.shape, df_patients.shape, df_pat_ep.shape

df_pat_ep['dateEqual']=df_pat_ep['EPISODE_DATE']==df_pat_ep['epDateInicial']
display(sum(df_pat_ep['dateEqual'].tolist()))
df_pat_ep.head(10)
display(sum(df_pat_ep['mmseInicial']<24))'''

In [ ]:
df_new.head(10)

In [ ]:
#### Calculate differnce beween patient API & Ahmad data
df_diff = df_new[['patient_id']]

df_patient_api = pd.read_csv(folder_path+'patients.csv')

df_diff['patient_id_1'] = df_patient_api['patient_id']

df_diff.head(10)

In [ ]:
set_ahmad = set(df_new['patient_id'].tolist()) 
df_patient_api = pd.read_csv(folder_path+'patients_df.csv') 
set_api = set(df_patient_api['patient_id'].tolist()) 
print (set_ahmad.difference(set_api)) 
print (sorted(set_api.difference(set_ahmad)))
list_val= sorted(set_api.difference(set_ahmad))

df_patient_api[df_patient_api['patient_id'].isin(list_val)].head(10)

In [ ]:
# Take Some other features from API
df_patient_api = pd.read_csv(folder_path+'patients.csv')
display(df_patient_api.head(10))
df_patient_api = df_patient_api[['patient_id', 'gender', 'dementia', 'smoker', 'alcohol', 'education',
       'bmi', 'weight', 'apoe']]
display(df_patient_api.head(10))
display(df_new.head(10))
df_patient_new = df_patient_api.merge(df_new, on=['patient_id'], how='inner')
df_patient_new.head(10)

In [ ]:
df_patient_new.to_csv(folder_path+'patients_new.csv', index=False)

In [ ]:
def removeNANvalues(lst):
    return lst[~pd.isnull(lst)]

comorbidity_cui_lst = list(set([y for x in removeNANvalues(df_patient_new[[col for col in df_patient_new.columns if 'Comorbidity_cui' in col]].values.flatten()) for y in x]))
medication_cui_lst = list(set([y for x in removeNANvalues(df_patient_new[[col for col in df_patient_new.columns if 'Medication_cui' in col]].values.flatten()) for y in x]))

with open(folder_path+'comorbidity_cui_lst.pkl', 'wb') as f:
    pickle.dump(comorbidity_cui_lst, f)
    
with open(folder_path+'medication_cui_lst.pkl', 'wb') as f:
    pickle.dump(medication_cui_lst, f)